<a href="https://colab.research.google.com/github/DeokwonWang/Content_Recommend_algorithm/blob/main/Recommend_spotlight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 설치

공식 문서 페이지에는 Conda로 Spotlight를 설치하기 위한 명령이 제공되지만 다음 명령을 사용하여 Git에서 직접 라이브러리를 설치하는 것이 좋습니다. 특히 Python 3.7이 있는 경우 더욱 그렇습니다.

In [2]:
!pip install git+https://github.com/maciejkula/spotlight.git #egg=spotlight 

  Cloning https://github.com/maciejkula/spotlight.git to /tmp/pip-req-build-j4cu79at
  Running command git clone -q https://github.com/maciejkula/spotlight.git /tmp/pip-req-build-j4cu79at
  Created wheel for spotlight: filename=spotlight-0.1.6-cp37-none-any.whl size=33933 sha256=dd1cb84afe771fd42d3cba3e3777532f5f4a051380b40fff08bc64c2f8ef8128
  Stored in directory: /tmp/pip-ephem-wheel-cache-0jxq5cwm/wheels/0a/33/c8/e8510ea648aaacf6031e128dfa92bcd3750f02db2aaf0922fe
Successfully built spotlight


# 영화 추천 시스템 구축

이 실제 사례에서 저는 Kaggle에서 제공되는 The Movies Dataset의 데이터를 사용하기로 결정했습니다. 이 데이터 세트에는 27만 명의 사용자가 45,000편의 영화에 대해 2,600만 건의 시청률을 기록한 파일이 포함되어 있습니다. 이 예에서는 데이터셋의 ratings_small 파일을 사용했는데, 여기에는 사용자 700명의 rating_small 등급 하위 집합이 포함되어 있습니다. 전체 데이터셋은 교육하는 데 훨씬 더 오랜 시간이 걸리지만 강력한 GPU가 장착된 시스템이 있는 경우 이 데이터셋을 사용해 볼 수 있습니다. 이 실제 예제의 전체 코드는 GitHub에서 확인할 수 있습니다.


# 기본 라이브러리 가져오기

아래 코드에서는 대부분의 데이터 과학 프로젝트에서 사용하는 기본 라이브러리를 간단하게 가져왔습니다.

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# 데이터 읽기

아래 코드에서는 세 가지 데이터 파일을 읽습니다.



*   ratings_small.csv — 다양한 사용자와 영화에 대한 등급 데이터가 포함되어 있습니다.
*   movies_movies.csv — 데이터 세트에 포함된 45,000편의 모든 영화에 대한 메타데이터가 포함되어 있습니다.
*   links.csv — 이 데이터를 동영상 메타데이터와 결합할 때 각 동영상을 조회하는 데 사용할 수 있는 ID를 포함합니다.









In [5]:
ratings_data = pd.read_csv('/content/drive/MyDrive/moviedata/ratings_small.csv')
metadata = pd.read_csv('/content/drive/MyDrive/moviedata/movies_metadata.csv')
links_data = pd.read_csv('/content/drive/MyDrive/moviedata/links.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
ratings_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  int64  
 1   movieId    100004 non-null  int64  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [12]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [13]:
links_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45843 entries, 0 to 45842
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  45843 non-null  int64  
 1   imdbId   45843 non-null  int64  
 2   tmdbId   45624 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 1.0 MB


다음 매핑 및 열 관계에 유의하십시오.

1. links_data의 movieId 열은 ratings_data의 movieId 열에 매핑됩니다.
2. links_data의 imdbId 열은 메타데이터의 imdb_id 열에 매핑됩니다.

# 메타데이터 전처리

이 다음 섹션에서는 아래 나열된 단계를 사용하여 데이터를 사전 처리했습니다. 자세한 내용은 GitHub에 대한 내 주피터 노트북과 Kaggle에 대한 데이터셋 설명을 참조하십시오.

1. imdb_id가 null인 메타데이터 데이터 프레임에서 행을 제거했습니다.
2. 람다 함수를 적용하여 메타데이터에서 imdb_id 열의 각 요소를 int로 변환합니다.
3. imbd_id 및 imdbId 열의 데이터 프레임을 각각 결합하여 메타데이터와 links_data를 병합합니다.

In [14]:
metadata = metadata[metadata['imdb_id'].notna()]

def remove_characters(string):
    
    return ''.join(filter(str.isdigit, string))
    
metadata['imdb_id'] = metadata['imdb_id'].apply(lambda x: int(remove_characters(str(x))))
full_metadata = pd.merge(metadata, links_data, left_on='imdb_id', right_on='imdbId')

위의 코드를 실행하면 동영상 ID만으로 동영상의 메타데이터를 검색할 수 있는 단일 데이터 프레임이 생성됩니다.

# Spotlight 상호 작용 데이터 세트 생성

Surprise와 마찬가지로 Spotlight에도 데이터 모델을 교육하기 위해 사용해야 하는 데이터셋 개체가 있습니다. 아래 코드에서는 다음 매개 변수를 제공하여 Interactions 개체를 생성했는데, 이 매개 변수는 모두 Numpy 어레이여야 합니다.



*   user_ids — 등급 데이터의 사용자 ID
*   Item_ids — 등급 데이터의 항목 ID
*   ratings — 등급 데이터에서 해당 등급.
*   timestamps(선택 사항) - 각 사용자/항목 상호 작용에 대한 타임스탬프입니다.







In [15]:
from spotlight.interactions import Interactions

dataset = Interactions(user_ids=ratings_data['userId'].values,
                       item_ids=ratings_data['movieId'].values,
                       ratings=ratings_data['rating'].values,
                       timestamps=ratings_data['timestamp'].values)

# 매트릭스 인수 분해 모델 교육

이제 데이터 세트가 생성되었으므로 아래 나온 것처럼 Spotlight의 명시적 요인화 모델 모듈을 사용하여 딥러닝 기반 매트릭스 인수화 모델을 교육할 수 있습니다.

In [16]:
from spotlight.cross_validation import random_train_test_split
from spotlight.evaluation import rmse_score
from spotlight.factorization.explicit import ExplicitFactorizationModel
train, test = random_train_test_split(dataset)

model = ExplicitFactorizationModel(n_iter=10)
model.fit(train, verbose=True)
rmse = rmse_score(model, test)
print('RMSE = ', rmse)

Epoch 0: loss 4.41037665080326
Epoch 1: loss 0.850391627648413
Epoch 2: loss 0.5401309328729595
Epoch 3: loss 0.38908267848898637
Epoch 4: loss 0.31060360180538243
Epoch 5: loss 0.2702022812253189
Epoch 6: loss 0.24821259432646425
Epoch 7: loss 0.2361675441663728
Epoch 8: loss 0.22643006715580907
Epoch 9: loss 0.21836985694950678
RMSE =  1.115788442159355


# 인수분해 모델을 사용하여 동영상 권장 사항 생성

이제 매트릭스 인수 분해 모델을 교육했으므로 이 모델을 사용하여 권장 동영상을 생성할 수 있습니다. 예측 방법은 단일 사용자 ID 또는 일련의 사용자 ID를 사용하고 데이터 세트의 각 동영상 항목에 대해 예측 등급 또는 "점수"를 생성합니다.

In [17]:
model.predict(user_ids=1)

array([0.39230758, 1.4769994 , 1.8814254 , ..., 0.33759582, 0.3539653 ,
       2.0365305 ], dtype=float32)

예측 방법의 출력은 각각 데이터셋의 항목(이 경우 동영상)에 대한 예측 등급 또는 점수에 해당하는 값의 배열입니다.

예측 방법의 출력을 실제 영화 추천으로 변환하기 위해 아래에 몇 가지 유틸리티 기능을 만들었습니다.

In [18]:
"""
Utility functions for generating movie recommendations using matrix factorization models
"""

def get_metadata(movie_id, metadata):
    """
    Retrieves the metadata for a movie given the movie ID
    """
    movie_data = metadata[metadata['movieId'] == movie_id]
    return movie_data[['original_title', 'release_date', 'genres']].to_dict(orient='records')

def recommend_movies(user_id, metadata, model, n_movies=5):
    """
    Recommends movies for user using a matrix factorization model.
    """ 
    pred = model.predict(user_ids=user_id)
    indices = np.argpartition(pred, -n_movies)[-n_movies:]
    best_movie_ids = indices[np.argsort(pred[indices])]
    
    return [get_metadata(movie_id + 1, metadata) for movie_id in best_movie_ids]

아래와 같이 recommend_movies 기능을 호출하여 특정 ID의 사용자에게 권장되는 동영상을 생성할 수 있습니다.

In [19]:
recommend_movies(1, full_metadata, model)

[[{'genres': "[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]",
   'original_title': "St. Elmo's Fire",
   'release_date': '1985-06-20'}],
 [{'genres': "[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name': 'Drama'}]",
   'original_title': 'The Cook, the Thief, His Wife & Her Lover',
   'release_date': '1989-09-10'}],
 [{'genres': "[{'id': 27, 'name': 'Horror'}, {'id': 878, 'name': 'Science Fiction'}]",
   'original_title': 'Bride of Frankenstein',
   'release_date': '1935-04-21'}],
 [{'genres': "[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]",
   'original_title': '喜宴',
   'release_date': '1993-08-04'}],
 [{'genres': "[{'id': 10402, 'name': 'Music'}, {'id': 18, 'name': 'Drama'}]",
   'original_title': 'Swing Kids',
   'release_date': '1993-03-05'}]]

이 기능을 호출하면 각 권장 동영상의 메타데이터가 포함된 사전 목록이 포함된 다음 출력이 생성됩니다.